## Import Libraries

In [1]:
import numpy as np
from scipy.stats import norm

## Define Black Scholes

In [2]:
r = 0.07
S = 100 
K = 100
T = 250 #time period days. needs to be less than N
N = 250 #number of trading days
Simulations = 5000
sigma = .2

In [3]:
def blackScholes(r, S, K, T, N, sigma, type="c"):
    """Calculate BS price of call/put. Adapted from https://quantpy.com.au/black-scholes-model/implementing-black-scholes-option-pricing/. 
    r = risk free rate of return
    S = current stock price
    K = Strike Price
    T = time to maturity
    N = Number of trading days in a year
    sigma = standard deviation of log returns (volatility)
    """
    T = T/N
    d1 = (np.log(S/K) + (r + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            price = S*norm.cdf(d1, 0, 1) - K*np.exp(-r*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            price = K*np.exp(-r*T)*norm.cdf(-d2, 0, 1) - S*norm.cdf(-d1, 0, 1)
        return price
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")

## Define Brownian Motion

In [5]:
def brownian_motion(r, S, K, T, N, sigma, type="c"):
    """Calculate BS price of call/put. Adapted from https://www.youtube.com/watch?v=Cb-GwN6jhNE
    r = risk free rate of return
    S = current stock price
    K = Strike Price
    T = time to maturity
    N = Number of trading days in a year
    sigma = standard deviation of log returns (volatility)
    """
    dt = (T/N)/N
    drift_coef = (r - 0.5 * sigma**2) * dt
    Z = np.random.normal(0, 1, (Simulations, N))
    right_hand_side = sigma * np.sqrt(dt)
    Prices = np.zeros((Simulations, T))
    Prices[:,0] += S
    for i in range(1, T):
        Prices[:,i] = Prices[:,i-1] * np.exp(drift_coef + right_hand_side * Z[:,i-1])
    try:
        if type == "c":
            c = Prices[:,-1] - K
            for i in range(len(c)):
                if c[i] < 0:
                    c[i] = 0
                else: 
                   c[i] = c[i] 
            payoff_call = np.mean(c)
            call = payoff_call*np.exp(-r*T/N)
            return call
        elif type == "p":
            p = K - Prices[:,-1] 
            for i in range(len(p)):
                if p[i] < 0:
                    p[i] = 0
                else: 
                   p[i] = p[i] 
            payoff_put = np.mean(p)
            put = payoff_put*np.exp(-r*T/N)
            return put
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")
 


## Compare Brownian Motion with Black Scholes

### Call Option

In [6]:
brownian_motion(r, S, K, T, N, sigma, type="c") 

11.585832417399258

In [7]:
blackScholes(r, S, K, T, N, sigma, type="c") 

11.541470170672412

### Put Option

In [8]:
brownian_motion(r, S, K, T, N, sigma, type="p") 

4.692094154207588

In [9]:
blackScholes(r, S, K, T, N, sigma, type="p") 

4.780852161267234